In [1]:
import torch
from torch import nn

In [11]:
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,3),nn.ReLU(),nn.Linear(3,1))
X=torch.rand(size=(2,4))
net(X)

tensor([[-0.5132],
        [-0.5195]], grad_fn=<AddmmBackward0>)

In [19]:
X,print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2947, -0.2525, -0.1051, -0.2648, -0.0816,  0.1219, -0.3384, -0.1254],
        [ 0.2486, -0.1987, -0.2989,  0.1334, -0.3271, -0.2164, -0.1493, -0.3097],
        [ 0.2249,  0.1312,  0.0759, -0.3254,  0.0879, -0.1533,  0.0591, -0.1869]])), ('bias', tensor([0.0198, 0.0163, 0.2073]))])


(tensor([[0.2441, 0.3490, 0.7298, 0.8838],
         [0.8737, 0.6903, 0.1954, 0.4962]]),
 None)

In [23]:
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([3, 8])) ('2.bias', torch.Size([3])) ('4.weight', torch.Size([1, 3])) ('4.bias', torch.Size([1]))


In [30]:
net.state_dict()['2.weight'].data


tensor([[ 0.2947, -0.2525, -0.1051, -0.2648, -0.0816,  0.1219, -0.3384, -0.1254],
        [ 0.2486, -0.1987, -0.2989,  0.1334, -0.3271, -0.2164, -0.1493, -0.3097],
        [ 0.2249,  0.1312,  0.0759, -0.3254,  0.0879, -0.1533,  0.0591, -0.1869]])

In [32]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                         nn.Linear(8,4),nn.ReLU())
def block2():
    net=nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1())
    return net

rgnet=nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [34]:
rgnet[0][0][0].weight.data

tensor([[-0.4685,  0.3036,  0.4363,  0.2000],
        [ 0.2382,  0.0945, -0.0775,  0.4470],
        [ 0.1262,  0.0683,  0.1765, -0.2977],
        [-0.3620, -0.3652,  0.1775,  0.1875],
        [-0.2608, -0.1693,  0.1493, -0.2715],
        [-0.2127, -0.0243,  0.1648,  0.1160],
        [ 0.1343,  0.0385, -0.4220, -0.0825],
        [ 0.0292, -0.1485,  0.1165,  0.1347]])

In [42]:
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([ 0.0052, -0.0049, -0.0021, -0.0116]), tensor(0.))

In [43]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [50]:
nn.init.constant_(net[0].weight,42),nn.init.xavier_uniform_(net[2].weight)
[(name,val) for name,val in net.named_parameters()]

[('0.weight',
  Parameter containing:
  tensor([[42., 42., 42., 42.],
          [42., 42., 42., 42.],
          [42., 42., 42., 42.],
          [42., 42., 42., 42.],
          [42., 42., 42., 42.],
          [42., 42., 42., 42.],
          [42., 42., 42., 42.],
          [42., 42., 42., 42.]], requires_grad=True)),
 ('0.bias',
  Parameter containing:
  tensor([0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)),
 ('2.weight',
  Parameter containing:
  tensor([[ 0.7001,  0.1961, -0.1171, -0.1157,  0.7302,  0.0582, -0.0231, -0.2691],
          [-0.4763,  0.6837,  0.4123, -0.6394, -0.5517, -0.3025, -0.6500, -0.3842],
          [-0.2024,  0.5203, -0.4632, -0.2551, -0.7007, -0.0167,  0.4860, -0.3722]],
         requires_grad=True)),
 ('2.bias',
  Parameter containing:
  tensor([0., 0., 0.], requires_grad=True)),
 ('4.weight',
  Parameter containing:
  tensor([[1., 1., 1.]], requires_grad=True)),
 ('4.bias',
  Parameter containing:
  tensor([0.], requires_grad=True))]

In [51]:
share=nn.Linear(8,8)
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                  share,nn.ReLU(),
                  share,nn.ReLU(),
                  nn.Linear(8,1))

In [54]:
net[2].weight.data,net[2].weight.data[0]

(tensor([[-0.0818, -0.0307,  0.1169,  0.1259,  0.1910,  0.3404, -0.2367, -0.1646],
         [ 0.2913,  0.2734,  0.3110,  0.3159, -0.2864,  0.3011,  0.1717,  0.2176],
         [ 0.2264, -0.0256, -0.2207,  0.3417,  0.3219, -0.1769,  0.0041, -0.0359],
         [-0.2393,  0.3513,  0.0805, -0.0057,  0.2193, -0.1385, -0.2252,  0.1113],
         [ 0.0225,  0.1112, -0.2339,  0.1512,  0.2952, -0.0957, -0.0757, -0.1552],
         [-0.2563,  0.0234, -0.1865,  0.3160, -0.0643,  0.3270, -0.2952, -0.1985],
         [ 0.2066, -0.0891,  0.2912, -0.0707, -0.2795, -0.1327,  0.3109,  0.2682],
         [-0.2845,  0.1283,  0.1615, -0.0273, -0.2996, -0.3374, -0.2339,  0.0760]]),
 tensor([-0.0818, -0.0307,  0.1169,  0.1259,  0.1910,  0.3404, -0.2367, -0.1646]))

In [57]:
net[2].weight.data==net[4].weight.data,net[4].weight.data

(tensor([[True, True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True, True]]),
 tensor([[-0.0818, -0.0307,  0.1169,  0.1259,  0.1910,  0.3404, -0.2367, -0.1646],
         [ 0.2913,  0.2734,  0.3110,  0.3159, -0.2864,  0.3011,  0.1717,  0.2176],
         [ 0.2264, -0.0256, -0.2207,  0.3417,  0.3219, -0.1769,  0.0041, -0.0359],
         [-0.2393,  0.3513,  0.0805, -0.0057,  0.2193, -0.1385, -0.2252,  0.1113],
         [ 0.0225,  0.1112, -0.2339,  0.1512,  0.2952, -0.0957, -0.0757, -0.1552],
         [-0.2563,  0.0234, -0.1865,  0.3160, -0.0643,  0.3270, -0.2952, -0.1985],
         [ 0.2066, -0.0891, 

In [58]:
net[2].weight.data+=1

In [62]:
print(net[4].weight.data[:]),print(net[2].weight.data)

tensor([[0.9182, 0.9693, 1.1169, 1.1259, 1.1910, 1.3404, 0.7633, 0.8354],
        [1.2913, 1.2734, 1.3110, 1.3159, 0.7136, 1.3011, 1.1717, 1.2176],
        [1.2264, 0.9744, 0.7793, 1.3417, 1.3219, 0.8231, 1.0041, 0.9641],
        [0.7607, 1.3513, 1.0805, 0.9943, 1.2193, 0.8615, 0.7748, 1.1113],
        [1.0225, 1.1112, 0.7661, 1.1512, 1.2952, 0.9043, 0.9243, 0.8448],
        [0.7437, 1.0234, 0.8135, 1.3160, 0.9357, 1.3270, 0.7048, 0.8015],
        [1.2066, 0.9109, 1.2912, 0.9293, 0.7205, 0.8673, 1.3109, 1.2682],
        [0.7155, 1.1283, 1.1615, 0.9727, 0.7004, 0.6626, 0.7661, 1.0760]])
tensor([[0.9182, 0.9693, 1.1169, 1.1259, 1.1910, 1.3404, 0.7633, 0.8354],
        [1.2913, 1.2734, 1.3110, 1.3159, 0.7136, 1.3011, 1.1717, 1.2176],
        [1.2264, 0.9744, 0.7793, 1.3417, 1.3219, 0.8231, 1.0041, 0.9641],
        [0.7607, 1.3513, 1.0805, 0.9943, 1.2193, 0.8615, 0.7748, 1.1113],
        [1.0225, 1.1112, 0.7661, 1.1512, 1.2952, 0.9043, 0.9243, 0.8448],
        [0.7437, 1.0234, 0.8135, 1.31

(None, None)